In [53]:
import pandas as pd
import os 

In [54]:
from os import walk

f = []
for (dirpath, dirnames, filenames) in walk("data/"):
    f.extend(filenames)
    #print(f.extend(filenames))
filenames =f

In [55]:

grabs=pd.read_csv("grabs_v1.csv")
grabs2=pd.read_csv("grabs_v2.csv")

In [56]:
def find_file(fil, grabs, grabs2):
    id_grab=int(fil.split("_")[0])
    filt=grabs[grabs.id_grabacion==id_grab]
    cand1="Not found"
    cand2="Not found"
    
    if len(filt)>0:
        cand1=".".join(filt.iloc[0]["nombre_original"].split(".")[:-1])
        prefijo1=filt.iloc[0]["prefijo"]
    filt=grabs2[grabs2.id_grabacion==id_grab]
    if len(filt)>0:
        cand2=".".join(filt.iloc[0]["nombre_original"].split(".")[:-1])
        prefijo2=filt.iloc[0]["prefijo"]
    if len(cand2)<2: cand2="Not found"
    if len(cand1)<2: cand1="Not found"
    filf=fil
    #print(filf)
    #print(cand1)
    #print(cand2)
    #print(filf.find(cand1))
    #print(filf.find(cand2))
    exito=0
    if filf.find(cand1)>=0 and filf.find(cand2)==-1:
        exito=1
        return prefijo1
    if filf.find(cand1)==-1 and filf.find(cand2)>=0:
        exito=1
        return prefijo2
        
    #print(exito)

In [57]:
dict_files={}
for f in filenames:
    dict_files[f]=find_file(f, grabs, grabs2)

In [58]:
bt=pd.read_csv("base_todos_2.csv")

In [59]:
", ".join(bt.columns)

'Unnamed: 0, Unnamed: 0.1, id_estudiante, tel_mas_confiable, id_tutor, prefijo, completado, completo_mate, completo_bienestar, completo_opinion, completo_español, completo_español_inicial, completo_mate_inicial, tratamiento, score_mat_inicial_x, score_mat_inicial_abiertas_x, score_mat_inicial_mult_x, grado_x, score_mat_inicial_y, score_mat_inicial_abiertas_y, score_mat_inicial_mult_y, grado_y, score_mat_inicial, score_mat_inicial_abiertas, score_mat_inicial_mult, grado, score_mat_final, score_mat_final_abiertas, score_cree, score_mat_final_mult, Unnamed: 0_x, score_esp_final, Por favor escribe un párrafo sobre quién es tu modelo a seguir y porque.\xa0, Por favor escribe un par sobre que quieres hacer cuando seas grande., Por favor escribe un par sobre que quieres hacer cuando seas grande.\nPor ejemplo\nCuando sea grande voy a tener un gato y un perro y voy a vivir en la ciudad de Tijuana cerca de mis papás y también voy a estudiar la carrera de Computación porque quiero ayudar a mi pap

In [60]:
#base you can use
#prefijo: group kid was assigned to, matches with values on dict_files
#ChangeMathScore: change in math score
#score_mat_inicial: initial math score (raw)
#score_mat_final: final math score (raw)
#tel_mas_confiable: if >0, it a class identifier
#MathScore: Final math score (standardized at grade level)
#std_mat_inicial: Initial math score (standardized at grade level)
#tratamiento: 1: control, 2: math, 3: math + emotional learning, 4: math

some_vars=bt[~pd.isna(bt.score_mat_final)][["ChangeMathScore", "score_mat_inicial", "score_mat_final", "tel_mas_confiable", "MathScore", "std_mat_inicial", "tratamiento", "prefijo"]]


In [64]:
prefijo_to_file = {v:k[:-5] for k,v in dict_files.items()}
some_vars['file'] = some_vars['prefijo'].map(prefijo_to_file)

In [66]:
some_vars.to_csv('reponse_var.csv', index=False)